In [1]:
import json
import onnx
import torch
import numpy as np
import pandas as pd
from torch import nn

In [2]:
class ExampleModel(nn.Module):

    def __init__(self):
        super(ExampleModel, self).__init__()

    def forward(self, u, U, X):
        U1 = torch.concat((U, u))
        U1 = U1[1:, :]
        x = X[-1:, :] + torch.sum(u)
        X1 = torch.concat((X, x))
        X1 = X1[1:, :]
        return x, X1, U1

In [3]:
FEATURES = 5
TARGETS = 3
T = 10
# Create three tensors
u = torch.ones((1, FEATURES))
U = torch.ones((T - 1, FEATURES)) * torch.arange(T, 1, -1).unsqueeze(1)
X = torch.ones(T, TARGETS)

# Create the model
model = ExampleModel()

# Run the model
output = model(u, U, X)
print(output)


(tensor([[6., 6., 6.]]), tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [6., 6., 6.]]), tensor([[9., 9., 9., 9., 9.],
        [8., 8., 8., 8., 8.],
        [7., 7., 7., 7., 7.],
        [6., 6., 6., 6., 6.],
        [5., 5., 5., 5., 5.],
        [4., 4., 4., 4., 4.],
        [3., 3., 3., 3., 3.],
        [2., 2., 2., 2., 2.],
        [1., 1., 1., 1., 1.]]))


In [4]:
model_name = "example4"
# Set the model to evaluation mode
model.eval()

# Save the model in ONNX format
torch.onnx.export(
    model,
    (u, U, X),
    f"{model_name}.onnx",
    verbose=True,
    input_names=["u", "U", "X"],
    output_names=["x", "X1", "U1"],
)

# Load the model
onnx_model = onnx.load(f"{model_name}.onnx")

# Check the model
onnx.checker.check_model(onnx_model)

# Add description to the model
onnx_model.graph.doc_string = "Example to test FMU with local variables."

# Add metadata to the model
onnx_model.producer_name = "ExampleModel"
onnx_model.producer_version = "0.0.1"
onnx_model.domain = "example"
onnx_model.model_version = 1

# Save the model
onnx.save(onnx_model, f"{model_name}.onnx")


Exported graph: graph(%u : Float(1, 5, strides=[5, 1], requires_grad=0, device=cpu),
      %U : Float(9, 5, strides=[5, 1], requires_grad=0, device=cpu),
      %X : Float(10, 3, strides=[3, 1], requires_grad=0, device=cpu)):
  %/Concat_output_0 : Float(10, 5, strides=[5, 1], requires_grad=0, device=cpu) = onnx::Concat[axis=0, onnx_name="/Concat"](%U, %u), scope: __main__.ExampleModel:: # /tmp/ipykernel_9061/1201402997.py:7:0
  %/Constant_output_0 : Long(1, strides=[1], requires_grad=0, device=cpu) = onnx::Constant[value={0}, onnx_name="/Constant"](), scope: __main__.ExampleModel:: # /tmp/ipykernel_9061/1201402997.py:8:0
  %/Constant_1_output_0 : Long(1, strides=[1], requires_grad=0, device=cpu) = onnx::Constant[value={1}, onnx_name="/Constant_1"](), scope: __main__.ExampleModel:: # /tmp/ipykernel_9061/1201402997.py:8:0
  %/Constant_2_output_0 : Long(1, strides=[1], requires_grad=0, device=cpu) = onnx::Constant[value={9223372036854775807}, onnx_name="/Constant_2"](), scope: __main__.Exa

## Generating model description

Create and save the model description to be provided to ONNX2FMU.

In [5]:
model_description = {
    "name": "example2",
    "description": "Example to test FMU with local variables.",
    "FMIVersion": "2.0",
    "inputs": [
        {
            "name": "u",
            "description": "A vector of control variables at time t."
        },
    ],
    "outputs": [
        {
            "name": "x",
            "description": "The state of the system at time t+1."
        }
    ],
    "locals": [
        {
            "nameIn": "X",
            "nameOut": "X1",
            "description": "The history of states from t-N to t."
        },
        {
            "nameIn": "U",
            "nameOut": "U1",
            "description": "The history of control variables frmo t-N to t-1."
        }
    ]
}

# Save model description
with open(f"{model_name}Description.json", "w", encoding="utf-8") as f:
    json.dump(model_description, f, indent=4)

## Generating input file and output for testing

In [6]:
time_steps = 100
U_hist = np.ones((time_steps, FEATURES)) * np.arange(time_steps)[:, None] + 1
columns = [f"u_0_{i}" for i in range(FEATURES)]
df = pd.DataFrame(
    data=U_hist,
    columns=columns,
    index=pd.Index(data=np.arange(time_steps), name='time')
).to_csv(f"input.csv")